<a href="https://colab.research.google.com/github/snpushpi/Differential-Privacy-in-Split-Learning/blob/main/SplitNN_RON_GAUSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import sys
import numpy as np 
import copy

! pip install ipython-autotime
%load_ext autotime

! pip install ipython-autotime
%load_ext autotime
print(f"Python: {sys.version}")
print(f"Pytorch: {torch.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print(f'GPU: {torch.cuda.current_device()}, {torch.cuda.device_count()}, {torch.cuda.get_device_name(0)}, {torch.cuda.is_available()}')
else: print(f'Device: cpu')

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Python: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
Pytorch: 1.6.0+cu101
Device: cpu
time: 5.18 s


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 24.2 s


In [4]:
mnist_data_path = '/content/drive/My Drive/archive'

time: 835 µs


In [35]:
transform = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))])

# load training set 
mnist_trainset = torchvision.datasets.MNIST(mnist_data_path, train=True, transform=transform, download=True)
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, shuffle=True)

# load test set
mnist_testset = torchvision.datasets.MNIST(mnist_data_path, train=False, transform=transform, download=True)
mnist_test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=128, shuffle=True)

from torch.utils.data.sampler import SubsetRandomSampler

total_size = len(mnist_trainset)

split1 = total_size // 4
split2 = split1*2
split3 = split1*3

indices = list(range(total_size))

alice_idx = indices[:split1]
bob_idx = indices[split1:split2]
mike_idx = indices[split2:split3]
rose_idc = indices[split3:]

alice_sampler = SubsetRandomSampler(alice_idx)
bob_sampler = SubsetRandomSampler(bob_idx)
mike_sampler = SubsetRandomSampler(mike_idx)
rose_sampler = SubsetRandomSampler(rose_idc)


alice_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=alice_sampler)
bob_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=bob_sampler)
mike_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=mike_sampler)
rose_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=rose_sampler)

data_loaders = [alice_loader, bob_loader, mike_loader, rose_loader ]

print(f'Data at alice: {len(alice_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at bob: {len(bob_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at mike: {len(mike_sampler)} \t Batches: {len(mike_loader)}')
print(f'Data at rose: {len(rose_sampler)} \t Batches: {len(rose_loader)}')

Data at alice: 15000 	 Batches: 118
Data at bob: 15000 	 Batches: 118
Data at mike: 15000 	 Batches: 118
Data at rose: 15000 	 Batches: 118
time: 133 ms


In [36]:
len(rose_loader)

118

time: 2.42 ms


In [133]:
client_model = torch.nn.Sequential(
                torch.nn.Conv2d(1, 32, kernel_size=5, padding=0, stride=1),  
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
                torch.nn.Conv2d(32, 32, kernel_size=5, padding=0, stride=1),  
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
                )

# send a copy to each data holder
alice_model = copy.deepcopy(client_model)
bob_model = copy.deepcopy(client_model)
mike_model = copy.deepcopy(client_model)
rose_model = copy.deepcopy(client_model)

# keep server copy at Server
server_model = torch.nn.Sequential(
                torch.nn.Flatten(),
                torch.nn.Linear(288,256),
                torch.nn.ReLU(),
                torch.nn.Linear(256, 10),  
                # torch.nn.Softmax(dim=-1),
        )


models = [alice_model, bob_model, mike_model, rose_model]

# Create optimizers for each model
optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]

#def cor_loss(x, split):
#  retun value

#def total_loss(cor_loss, celoss):
#  return x*cor_loss + y*celoss

time: 27.2 ms


In [121]:
server_optimizer = optim.Adam(server_model.parameters(), lr=0.01)
server_loss = nn.CrossEntropyLoss()

time: 1.09 ms


In [122]:
import numpy as np
from sklearn.preprocessing import normalize
import math

time: 999 µs


In [123]:
def pre_processing(dataset,epsilon_mu):
    '''Input Numpy Matrix of dimension (m,n) and privacy param epsion '''
    pre_normalized_dataset = normalize(dataset,axis=0)
    #deriving dp mean
    avg = np.mean(pre_normalized_dataset,axis=1)
  
    m,n = pre_normalized_dataset.shape
    avg = avg.reshape((m,1))
    loc_param = 2*math.sqrt(m)/(n*epsilon_mu)
    laplace_noise = np.random.laplace(0,loc_param,(m,1))
    mu_dp = avg+laplace_noise
    #centerize the data
    mat_1 = np.ones((1,n))
    centralized_dataset = dataset - np.matmul(mu_dp,mat_1)
    final_dataset = normalize(centralized_dataset,axis=0)
    return final_dataset,mu_dp


time: 7.28 ms


In [124]:
def ron_projection(pre_processed_dataset,p):
    m,n = pre_processed_dataset.shape
    A = np.random.uniform(0,1,(m,m))
    Q,R = np.linalg.qr(A, mode='complete')
    #constructing a Ron Projection Matrix
    W = Q[:,:p]
    projected_data = np.matmul(np.transpose(W),pre_processed_dataset)
    return projected_data,W


time: 2.93 ms


In [125]:
def ron_gauss_for_supervised(dataset,training_label,a,dimension_p,epsilon_mu,epsilon_sigma):
    #label.shape = (n,1)
    preprocessed_data,dp_mu = pre_processing(dataset,epsilon_mu)
    ron_projected_data,W = ron_projection(preprocessed_data,dimension_p)
    p,n = ron_projected_data.shape
    augmented_ron = np.vstack((ron_projected_data,np.transpose(training_label)))
    sqp = math.sqrt(p)
    loc_param = (2*sqp+4*a*sqp+a*a)/(n*epsilon_sigma)
    laplace_noise = np.random.laplace(0,loc_param,(dimension_p+1,dimension_p+1))
    dp_cov = (1/n)*np.matmul(augmented_ron,np.transpose(augmented_ron))+laplace_noise
    synthesized_dp_data = np.random.multivariate_normal(np.zeros((dimension_p+1,)), dp_cov, size=(n,))
    return np.transpose(synthesized_dp_data)


time: 9.93 ms


In [126]:
def ron_gauss_extension_gmm(dataset,training_label,label_list,dimension_p,epsilon_mu,epsilon_sigma):
    #will be used for classification
    #training label shape = (n,1)
    new_filtered_dataset = []
    new_labels = []
    for label in label_list:
        labels = np.where(training_label==label)[0]
        n_c = len(labels)
        if n_c==0:
            continue
        X_label = dataset[:,labels]
        X_preprocessed,dp_mu = pre_processing(X_label,epsilon_mu)
        X_ron_projected,W = ron_projection(X_preprocessed,dimension_p)
        loc_param = 2*math.sqrt(dimension_p)/(n_c*epsilon_sigma)
        laplace_noise = np.random.laplace(0,loc_param,(dimension_p,dimension_p))
        dp_cov = (1/n_c)*np.matmul(X_ron_projected,np.transpose(X_ron_projected))+laplace_noise
        synthesized_dp_data = np.random.multivariate_normal(np.zeros((dimension_p,)), dp_cov, size=(n_c,))
        print(synthesized_dp_data.shape)
        if len(new_filtered_dataset)==0 :
            new_filtered_dataset = np.transpose(synthesized_dp_data)
            new_labels = np.full((n_c,1),label)
        else:
            new_filtered_dataset = np.hstack((new_filtered_dataset,np.transpose(synthesized_dp_data)))
            l = np.full((n_c,1),label)
            new_labels = np.vstack((new_labels,l))
    return new_filtered_dataset,new_labels


time: 14.3 ms


In [127]:
#processes the data itself and generates dp synthesized data from the dataloader part
imgs, lbls = [],[]
label_list = [0,1,2,3,4,5,6,7,8,9]
for i in range(4):
    imgsc,lblsc = next(iter(data_loaders[i]))
    imgsc = imgsc.reshape((128,784)).numpy()
    lblsc = lblsc.reshape((128,1)).numpy()
    imgsc = np.transpose(imgsc)
    dp_imgsc,dp_lblsc = ron_gauss_extension_gmm(imgsc,lblsc,label_list,625,0.6,1.4)
    dp_imgsc = np.transpose(dp_imgsc)
    dp_imgsc = torch.from_numpy(dp_imgsc)
    dp_lblsc = torch.from_numpy(dp_lblsc)
    dp_imgsc = torch.reshape(dp_imgsc,(128,1,25,25))
    dp_lblsc = torch.reshape(dp_lblsc,(128,))
    imgs.append(dp_imgsc)
    lbls.append(dp_lblsc)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not positive-semidefinite.


(11, 625)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not positive-semidefinite.


(12, 625)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not positive-semidefinite.


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not positive-semidefinite.


(12, 625)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not positive-semidefinite.


(18, 625)
(14, 625)
time: 12.7 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not positive-semidefinite.


In [128]:
lbls[0].shape

torch.Size([128])

time: 4.93 ms


In [129]:
imgs[0].shape

torch.Size([128, 1, 25, 25])

time: 2.25 ms


In [137]:
epochs = 7

for e in range(epochs):
  running_loss = 0

  # iterate based on batch numbers assuming it is unified acorss all clients
  # a more efficient solution is to assign the batch size at each client relative on its data size
  # this would guarantee less latency at the server side given that client-side training is parallaized 

  for i in range(len(data_loaders[0])):
    # assuming a gprc setup 
    for opt in optimizers:
      opt.zero_grad()
    
    server_optimizer.zero_grad()

    lst_of_vars = []

    for i in range(len(data_loaders)): 
      images, labels = imgs[i], lbls[i] 
      split_output = models[i](images.float()) 
      split_layer_output = split_output.clone().detach().requires_grad_(True)
      #cor_loss = (imgs, split_layer_output)

      server_output = server_model(split_layer_output)
      loss = server_loss(server_output, labels)
      lst_of_vars.append({'split_output': split_output, 'split_layer_output': split_layer_output, 'loss':loss})
    loss = 0  
    for i in range(len(data_loaders)):
      loss += lst_of_vars[i]['loss']

    avg_loss = loss / len(data_loaders)
    running_loss += avg_loss

    avg_loss.backward()

    for i in range(len(data_loaders)):
      split_gradients = lst_of_vars[i]['split_layer_output'].grad.clone().detach()
      lst_of_vars[i]['split_output'].backward(split_gradients)
    
    server_optimizer.step()
    
    for opt in optimizers:
      opt.step()

  print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(data_loaders[0])))


Epoch 1 - Training loss: 1.8920410871505737
Epoch 2 - Training loss: 0.8316487669944763
Epoch 3 - Training loss: 0.7194857001304626
Epoch 4 - Training loss: 0.6854436993598938
Epoch 5 - Training loss: 0.6883317232131958
Epoch 6 - Training loss: 1.1583611965179443
Epoch 7 - Training loss: 1.1578493118286133
time: 3min 11s


In [139]:
correct = 0
total = 0
with torch.no_grad():
    for data in mnist_test_loader:
        images, labels = data
        print(images.shape)
        outputs = bob_model(images)
        outputs = server_model(outputs)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the 10000 test images using SPLIT INFERENCE: %d %%' % (100 * correct / total))

torch.Size([128, 1, 28, 28])


RuntimeError: ignored

time: 75.7 ms
